In [82]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import sqlalchemy
import pymysql
from keys import *  



In [83]:

city = "Tokyo"
country = "JPN"

url = f"http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en"

response = requests.get(url)
if response.status_code != 200:
    print("❌ Error al obtener datos de OpenWeatherMap:", response.status_code)
else:
    print("Datos obtenidos correctamente.")
    data = response.json()



Datos obtenidos correctamente.


In [84]:
forecast_list = data.get('list', [])

times = []
temperatures = []
humidities = []
weather_statuses = []
wind_speeds = []
rain_volumes = []
snow_volumes = []

for entry in forecast_list:
    times.append(entry.get('dt_txt', np.nan))
    temperatures.append(entry.get('main', {}).get('temp', np.nan))
    humidities.append(entry.get('main', {}).get('humidity', np.nan))
    weather_statuses.append(entry.get('weather', [{}])[0].get('main', np.nan))
    wind_speeds.append(entry.get('wind', {}).get('speed', np.nan))
    rain_volumes.append(entry.get('rain', {}).get('3h', 0))
    snow_volumes.append(entry.get('snow', {}).get('3h', 0))


df = pd.DataFrame({
    'weather_datetime': times,
    'temperature': temperatures,
    'humidity': humidities,
    'weather_status': weather_statuses,
    'wind': wind_speeds,
    'rain_qty': rain_volumes,
    'snow': snow_volumes,
    'municipality_iso_country': f"{city},{country}"
})


df['weather_datetime'] = pd.to_datetime(df['weather_datetime'])

print("🔹 Vista previa del DataFrame:")
display(df.head())



🔹 Vista previa del DataFrame:


,weather_datetime,temperature,humidity,weather_status,wind,rain_qty,snow,municipality_iso_country
0,2025-11-03 06:00:00,17.38,37,Clouds,11.01,0.0,0,"Tokyo,JPN"
1,2025-11-03 09:00:00,16.61,39,Clouds,9.87,0.0,0,"Tokyo,JPN"
2,2025-11-03 12:00:00,15.15,40,Clouds,9.11,0.0,0,"Tokyo,JPN"
3,2025-11-03 15:00:00,13.00,43,Clear,7.69,0.0,0,"Tokyo,JPN"
4,2025-11-03 18:00:00,12.16,47,Clear,5.62,0.0,0,"Tokyo,JPN"


In [85]:
schema = "gans"
host = "127.0.0.1"
user = "root"
password = "Ua-xxx99" 
port = 3306

engine = sqlalchemy.create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}")


df.to_sql("weather_data", con=engine, if_exists="append", index=False)

print("✅ Datos insertados correctamente en MySQL (ciudad: Tokyo, JPN).")



✅ Datos insertados correctamente en MySQL (ciudad: Tokyo, JPN).


In [87]:
from sqlalchemy import text

with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM weather_data LIMIT 10;"))
    for row in result:
        print(row)




(81, datetime.datetime(2025, 10, 30, 0, 0), Decimal('12'), 67, 'Clouds', Decimal('5'), None, None, 'Tokyo,JPN')
(82, datetime.datetime(2025, 10, 30, 3, 0), Decimal('14'), 58, 'Clouds', Decimal('4'), None, None, 'Tokyo,JPN')
(83, datetime.datetime(2025, 10, 30, 6, 0), Decimal('16'), 45, 'Clear', Decimal('3'), None, None, 'Tokyo,JPN')
(84, datetime.datetime(2025, 10, 30, 9, 0), Decimal('18'), 42, 'Clear', Decimal('4'), None, None, 'Tokyo,JPN')
(85, datetime.datetime(2025, 10, 30, 12, 0), Decimal('16'), 55, 'Clouds', Decimal('4'), None, None, 'Tokyo,JPN')
(86, datetime.datetime(2025, 10, 30, 15, 0), Decimal('16'), 58, 'Clouds', Decimal('3'), None, None, 'Tokyo,JPN')
(87, datetime.datetime(2025, 10, 30, 18, 0), Decimal('15'), 50, 'Clouds', Decimal('4'), None, None, 'Tokyo,JPN')
(88, datetime.datetime(2025, 10, 30, 21, 0), Decimal('15'), 47, 'Clouds', Decimal('4'), None, None, 'Tokyo,JPN')
(89, datetime.datetime(2025, 10, 31, 0, 0), Decimal('16'), 45, 'Clouds', Decimal('5'), None, None, 'To